In [3]:
import pandas as pd

df = pd.read_csv('data/train.csv')
df.head(2)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200


In [4]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [5]:
agent = create_csv_agent(OpenAI(temperature=0), 
                         'data/train.csv', 
                         verbose=True)

In [6]:
agent

AgentExecutor(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x15498bac0>, verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x15498bac0>, verbose=False, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={'df': '   User_ID Product_ID Gender   Age  Occupation City_Category  \\\n0  1000001  P00069042      F  0-17          10             A   \n1  1000001  P00248942      F  0-17          10             A   \n2  1000001  P00087842      F  0-17          10             A   \n3  1000001  P00085442      F  0-17          10             A   \n4  1000002  P00285442      M   55+          16             C   \n\n  Stay_In_Current_City_Years  Marital_Status  Product_Category_1  \\\n0                          2               0                   3   \n1                          2               0                   

In [7]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [8]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: len(df)
Observation: 550068
Thought: I now know the final answer
Final Answer: 550068

> Finished chain.


'550068'

In [9]:
agent.run("how many people are female?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of people who are female
Action: python_repl_ast
Action Input: df[df['Gender'] == 'F'].count()
Observation: User_ID                       135809
Product_ID                    135809
Gender                        135809
Age                           135809
Occupation                    135809
City_Category                 135809
Stay_In_Current_City_Years    135809
Marital_Status                135809
Product_Category_1            135809
Product_Category_2             91530
Product_Category_3             37594
Purchase                      135809
dtype: int64
Thought: I now know the final answer
Final Answer: 135809

> Finished chain.


'135809'

In [10]:
agent.run("how many people have stayed more than 3 years in the city?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to find the people who have stayed more than 3 years in the city
Action: python_repl_ast
Action Input: df[df['Stay_In_Current_City_Years'] > '3'].shape[0]
Observation: 84726
Thought: I now know the final answer
Final Answer: 84726 people have stayed more than 3 years in the city.

> Finished chain.


'84726 people have stayed more than 3 years in the city.'

In [11]:
agent.run("how many people have stayed more than 3 years in the city and are female?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to find the people who meet the criteria
Action: python_repl_ast
Action Input: df[(df['Stay_In_Current_City_Years'] > '3') & (df['Gender'] == 'F')]
Observation:         User_ID Product_ID Gender    Age  Occupation City_Category  \
29      1000010  P00085942      F  36-45           1             B   
30      1000010  P00118742      F  36-45           1             B   
31      1000010  P00297942      F  36-45           1             B   
32      1000010  P00266842      F  36-45           1             B   
33      1000010  P00058342      F  36-45           1             B   
...         ...        ...    ...    ...         ...           ...   
549972  1005901  P00372445      F  36-45           7             C   
549981  1005911  P00372445      F  26-35          17             C   
550011  1005952  P00371644      F  46-50           1             C   
550065  1006036  P00375436      F  26-35          15       

'18596 people have stayed more than 3 years in the city and are female.'